In [1]:
#Set-up
import os
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import seaborn as sns
import warnings
from IPython.display import clear_output
import nibabel as nib

import torch
import torchvision.models as models
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn as nn
from torch.optim import lr_scheduler
import torch.nn.functional as F
import torch.utils.data as utils
import time
import copy

In [2]:
x = torch.load("../../../data/data.pt")
y = torch.load("../../../data/ages.pt")
n = x.shape[0]
print(x.shape,y.shape)

torch.Size([536, 3, 224, 224]) torch.Size([536])


In [3]:
fcp_x = torch.load("../../../data/fcp_data.pt")
fcp_y = torch.load("../../../data/fcp_ages.pt")
fcp_n = fcp_x.shape[0]
combined_x = torch.cat((x,fcp_x))
combined_y = torch.cat((y,fcp_y))
combined_y = torch.reshape(combined_y,(fcp_n + n,1))
N = combined_x.shape[0]
print(combined_x.shape,combined_y.shape)

torch.Size([1570, 3, 224, 224]) torch.Size([1570, 1])


In [4]:
#Normalizing the data (according to the stats provided) before feeding into the pretrained model
x_normalized = torch.Tensor(x.shape)
for i in range(n):
    x_normalized[i] = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225]).__call__(x[i,:,:])
#     fcp_x_normalized[i] = transforms.Normalize(mean=[0.485, 0.456, 0.406],
#                                      std=[0.229, 0.224, 0.225]).__call__(fcp_x[i,:,:])
    

In [5]:
USE_GPU = True

dtype = torch.float32 # we will be using float throughout this tutorial

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print('using device:', device)

using device: cuda


In [6]:
y = torch.reshape(y,(n,1))

In [7]:
# dataloaders = {i: torch.utils.data.DataLoader(dataset, batch_size=4,
#                                              shuffle=True, num_workers=4)
#               for i in ['train', 'val']}
train_indices = [i for i in range(n) if (i % 5 == 0 or i % 5 == 2 or i % 5 == 4)]
val_indices = [i for i in range(n) if i % 5 == 3]
test_indices = [i for i in range(n) if i not in train_indices and i not in val_indices]

print("There are %d (%.2f) training data points, %d (%.2f) validation data points,\and %d (%.2f) test data points" %(len(train_indices),len(train_indices)/n,len(val_indices),\
                                                                                                                len(val_indices)/n,len(test_indices),len(test_indices)/n))
train_dataset = utils.TensorDataset(x_normalized[train_indices],y[train_indices])
val_dataset = utils.TensorDataset(x_normalized[val_indices],y[val_indices])      
dataloaders = {}
dataloaders['train'] = torch.utils.data.DataLoader(train_dataset, batch_size=4,
                                             shuffle=True,num_workers=4)
dataloaders['val'] = torch.utils.data.DataLoader(val_dataset, batch_size=4,
                                             shuffle=True,num_workers=4)

There are 322 (0.60) training data points, 107 (0.20) validation data points,nd 107 (0.20) test data points


In [8]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_offset = float('inf')

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_offset = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device,dtype = torch.float)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_offset += torch.sum(torch.abs(outputs - labels.data))
                
            if phase == 'train':
                epoch_loss = running_loss / (n * 0.6)
                epoch_offset = running_offset / (n * 0.6)
               
            else:
                epoch_loss = running_loss / (n * 0.2)
                epoch_offset = running_offset / (n * 0.2)
            
            print('{} Loss: {:.4f} Diff: {:.4f}'.format(
                phase, epoch_loss, epoch_offset))

            # deep copy the model
            if phase == 'val' and epoch_offset < best_offset:
                best_offset = epoch_offset
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Diff: {:4f}'.format(best_offset))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [13]:
model_ft = models.resnet101(pretrained=True)

num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 1)

model_ft = model_ft.to(device)

criterion = nn.MSELoss()

# Observe that all parameters are being optimized
# optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)
optimizer_ft = optim.Adam(model_ft.parameters(), lr=0.001)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [17]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=50)

Epoch 0/49
----------
train Loss: 125.5311 Diff: 9.2308
val Loss: 99.1176 Diff: 8.1358

Epoch 1/49
----------
train Loss: 138.9898 Diff: 9.6144
val Loss: 100.4619 Diff: 8.0238

Epoch 2/49
----------
train Loss: 103.7960 Diff: 8.0050
val Loss: 97.9270 Diff: 8.0415

Epoch 3/49
----------
train Loss: 110.8339 Diff: 8.4225
val Loss: 98.5248 Diff: 8.0560

Epoch 4/49
----------
train Loss: 133.0807 Diff: 8.9952
val Loss: 100.5171 Diff: 8.0231

Epoch 5/49
----------
train Loss: 110.7878 Diff: 8.4692
val Loss: 101.4549 Diff: 7.9981

Epoch 6/49
----------
train Loss: 121.5263 Diff: 8.8809
val Loss: 106.0599 Diff: 8.1464

Epoch 7/49
----------
train Loss: 118.8650 Diff: 8.8063
val Loss: 98.4922 Diff: 8.1197

Epoch 8/49
----------
train Loss: 111.3667 Diff: 8.4513
val Loss: 99.3333 Diff: 8.1445

Epoch 9/49
----------
train Loss: 128.4246 Diff: 9.1009
val Loss: 98.9834 Diff: 8.0222

Epoch 10/49
----------
train Loss: 100.3451 Diff: 7.9407
val Loss: 98.7751 Diff: 8.1253

Epoch 11/49
----------
trai

In [15]:
def check_error(data,labels,model):
    print('Checking error on test set')   
    total_abs_err = 0.0
    num_samples = data.shape[0]
    model.eval()  # set model to evaluation mode
    with torch.no_grad():
        data = data.to(device=device, dtype=dtype)  # move to device, e.g. GPU
        labels = labels.to(device=device,dtype=dtype)
        scores = model(data)
        total_abs_err += torch.abs(labels - scores).sum()
        mean_abs_err = float(total_abs_err) / num_samples
        print('Got error %.4f' % (mean_abs_err))

In [16]:
# print(x_normalized[test_indices,:].shape)
check_error(x_normalized[test_indices,:], y[test_indices],model_ft)

Checking error on test set
Got error 8.6252


In [41]:
torch.save(model_ft,"best_reg_model_res18.pth")